In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
import keras.backend as k
from keras.optimizers import RMSprop 
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Load data

In [2]:
car_x=[]
car_y=[]

# path = './Cars/BMWImages'
# path = './Cars/BMWImages'
# path = './Cars/BMWImages'
path = '.\\Cars\\BMWImages_30'

labels = listdir(path)

for labelName in labels:
    file_path = join(path,labelName)
    image_names = listdir(file_path)
    for aimage in image_names:
        image_path = join(file_path,aimage)
        train_image = load_img(image_path)  # RGB Image
        train_image = img_to_array(train_image)  # this is a Numpy array with shape (3, 150, 150)
        car_x.append(train_image)
        car_y.append(labelName)
        
car_x_ori= car_x.copy()
car_y_ori= car_y.copy()

In [3]:
len(car_x_ori)

1229

### Data preprocessing

In [4]:
car_x= car_x_ori
car_y= car_y_ori

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(car_y)
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
car_y = onehot_encoder.fit_transform(integer_encoded)

num_class = len(labels)
input_shape = car_x[0].shape

# X_train, X_test, Y_train, Y_test = train_test_split(car_x, car_y, test_size=0.2,shuffle = True,random_state=1)
car_x = np.asarray(car_x)
car_x = car_x.astype('float32')
car_x /= 255


X_train, X_test, Y_train, Y_test = train_test_split(car_x, car_y, test_size=0.2,shuffle = True)

## Inception V3 Model

In [5]:
def build_model(nb_classes, inputs_shape):
    k.clear_session()
    base_model = InceptionV3(weights='imagenet', include_top=False,input_shape=inputs_shape)
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
#     x = Dropout(0.5)(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
#     predictions = Dropout(0.5)(predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
#     for layer in base_model.layers:
#         layer.trainable = False
    
#     for index in range(-1,-4,-1):
#         base_model.layers[index] = True
    for layer in base_model.layers[:249]:
        layer.trainable = False
    for layer in base_model.layers[249:]:
        layer.trainable = True
    count = 0
    for layer in base_model.layers:
        if ('batch_normalizatio' in layer.name):
            count +=1
            layer.trainable =True
            if(count>=19):
                break

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

def compile(model,learn_rate):
    model.compile(optimizer=RMSprop(lr=learn_rate), loss='categorical_crossentropy', metrics=["accuracy"])
#     model.compile(optimizer=RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])
#     model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

### Train and evaluate

In [6]:
model = build_model(num_class, input_shape)
compile(model,0.0005)

# model.summary()

# for layer in model.non_trainable_weights:
#     print(layer.name)

# for layer in model.trainable_weights:
#     print(layer.name)

In [7]:
# len(model.layers)
len(model.trainable_weights)


# for layer in model.layers[:249]:
#     layer.trainable = False
# for layer in model.layers[249:]:
#     layer.trainable = True
# for layer in model.trainable_weights:
#     print(layer.name)

# for layer in model.layers[10:]:
#     if ('batch_normalizatio' in layer.name):
#         layer.trainable =True
# for layer in model.trainable_weights:
#     print(layer.name)


59

In [8]:
epochs = 10
# epochs = 30
# batch_size = 20
# batch_size = 200
batch_size = 100

history=model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs)


# history=model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_test, Y_test))
# score = model.evaluate(X_test, Y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/10
983/983 [==============================] - 355s 361ms/step - loss: 3.5265 - accuracy: 0.0814
Epoch 2/10
983/983 [==============================] - 291s 296ms/step - loss: 2.2501 - accuracy: 0.3845
Epoch 3/10
983/983 [==============================] - 290s 295ms/step - loss: 0.6859 - accuracy: 0.8525
Epoch 4/10
983/983 [==============================] - 290s 295ms/step - loss: 0.1185 - accuracy: 0.9868
Epoch 5/10
983/983 [==============================] - 291s 296ms/step - loss: 0.0420 - accuracy: 0.9959
Epoch 6/10
983/983 [==============================] - 295s 301ms/step - loss: 0.0204 - accuracy: 0.9980
Epoch 7/10
983/983 [==============================] - 291s 296ms/step - loss: 0.0331 - accuracy: 0.9939
Epoch 8/10
983/983 [==============================] - 291s 296ms/step - loss: 0.0345 - accuracy: 0.9939
Epoch 9/10
983/983 [==============================] - 290s 295ms/step - loss: 0.1197 - accuracy: 0.9705
Epoch 10/10
983/983 [==============================] - 290s 295m

In [ ]:
print(history.history.keys())


plt.plot(history.history['loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

plt.plot(history.history['accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()


# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# # plt.show()

In [9]:
score = model.evaluate(X_test, Y_test, verbose=0)
# score = model.evaluate(X_train, Y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 10.888925606642312
Test accuracy: 0.1463414579629898


In [10]:
model.save('./ModelBox/car_30Class_54.h5')


In [ ]:
# model.evaluate(X_train[0:10], Y_train[0:10], verbose=0)
# model.predict(X_train[1], Y_train[1], verbose=0)
# np.expand_dims(X_train[1], axis=3)


# model.evaluate(X_train[0:100], Y_train[0:100], verbose=0)
# model.evaluate(X_test[0:10], Y_test[0:10], verbose=0)
model.evaluate(X_train, Y_train, verbose=0)


## Resize Image

In [3]:
from PIL import Image, ImageOps
import os
import shutil
from IPython.display import display # to display images

import pandas as pd

# original_image = Image.open("path/to/image")
# size = (512, 512)
# fit_and_resized_image = ImageOps.fit(original_image, size, Image.ANTIALIAS)

In [23]:
# delta_w = desired_size - new_size[0]
# delta_h = desired_size - new_size[1]
# padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(elta_h//2))
# new_im = ImageOps.expand(im, padding)

# new_im.show()
car_path = './Cars'
car_class = pd.read_csv(os.path.join(car_path,'car_train_class.csv'))
# car_class.head()
class_50 = car_class.sort_values(by=['class'])

class_50 = class_50.reset_index()

In [ ]:
class_50.loc[class_50['class']==1]

In [47]:
count = 1
size=[200,200]
path = './Cars/cars_train'
smallPath = './Cars/BMWImages_30'
for class_num in range(1,31):
    tmp_df = class_50.loc[class_50['class']==class_num]
    tmp_path = os.path.join(smallPath,str(class_num-1))
    if(not os.path.exists( tmp_path)):
        os.mkdir(tmp_path)
    
    for index, row in tmp_df.iterrows():
        ori_path = os.path.join(path,row['fname'])
        new_path = os.path.join(tmp_path,row['fname'])
        original_image = Image.open(ori_path)
        fit_and_resized_image = ImageOps.fit(original_image, size, Image.ANTIALIAS)
        fit_and_resized_image.save(new_path)

In [48]:
# cars_train_path = './Cars/cars_train'

# original_image = Image.open(os.path.join(cars_train_path,car_class['fname'][0]))
# original_image

# # original_image = Image.open("./bmw10_ims/1.jpg")
# # # size = (512, 512)
# size = (300, 300)
# fit_and_resized_image = ImageOps.fit(original_image, size, Image.ANTIALIAS)

# # display(original_image)

# # fit_and_resized_image
# fit_and_resized_image.size

In [17]:
path = '.\\bmw10_ims'
smallPath = '.\\BMWImages'
labels_list = listdir(path)

size = (200, 200)

for labelName in labels_list:
    tmp_path = join(smallPath,labelName)
    if(not os.path.exists( tmp_path)):
        os.mkdir(tmp_path)

    ori_path = join(path,labelName)
    new_path = join(smallPath,labelName)
    
    image_names = listdir(ori_path)
    image_num =0
    randonIndex =  np.random.choice(len(image_names),20, replace=False)
    for ind in randonIndex:
        image_num +=1
        
        original_image = Image.open(join(ori_path,image_names[ind]))
        fit_and_resized_image = ImageOps.fit(original_image, size, Image.ANTIALIAS)
        fit_and_resized_image.save(join(new_path,image_names[ind]))

#         shutil.copyfile(join(ori_path,image_names[ind]),join(new_path,image_names[ind]))
#         if(image_num==10):
#             break

# ========================================================

### Data preprocessing

In [49]:
from sklearn.model_selection import train_test_split

In [56]:
X_car=[]
Y_car=[]

# path_train = '.\\LatestData\Training'
# smallPath = '.\\BMWImages'
smallPath = '.\\Cars\\BMWImages_30'
labels = os.listdir(smallPath)

for labelName in labels:
    file_path = os.path.join(smallPath,labelName)
    image_names = os.listdir(file_path)
    for aimage in image_names:
        image_path = os.path.join(file_path,aimage)
        train_image = load_img(image_path)  # RGB Image

        train_image = img_to_array(train_image)  # this is a Numpy array with shape (3, 150, 150)
        X_car.append(train_image)
        Y_car.append(labelName)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y_car)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y_car = onehot_encoder.fit_transform(integer_encoded)

num_class = len(labels)
input_shape = X_car[0].shape

X_car = np.asarray(X_car)

X_car = X_train.astype('float32')
X_car /= 255

X_train, X_test, Y_train, Y_test = train_test_split(X_car, Y_car, test_size=0.2,shuffle = True,random_state=34)




NameError: name 'load_img' is not defined

In [ ]:
X_train=[]
Y_train=[]

# path_train = '.\\LatestData\Training'
# smallPath = '.\\BMWImages'
smallPath = '.\\Cars\\BMWImages_30'
labels = listdir(path_train)

for labelName in labels:
    file_path = join(path_train,labelName)
    image_names = listdir(file_path)
    for aimage in image_names:
        image_path = join(file_path,aimage)
        train_image = load_img(image_path)  # RGB Image

        train_image = img_to_array(train_image)  # this is a Numpy array with shape (3, 150, 150)
        X_train.append(train_image)
        Y_train.append(labelName)

X_test=[]
Y_test=[]
path_test = '.\\LatestData\Test'
labels = listdir(path_train)
for labelName in labels:
    file_path = join(path_test,labelName)
    image_names = listdir(file_path)
    for aimage in image_names:
        image_path = join(file_path,aimage)
        train_image = load_img(image_path)  # RGB Image

        train_image = img_to_array(train_image)  # this is a Numpy array with shape (3, 150, 150)
        X_test.append(train_image)
        Y_test.append(labelName)        

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y_train)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y_train = onehot_encoder.fit_transform(integer_encoded)

integer_encoded = label_encoder.fit_transform(Y_test)
Y_test = onehot_encoder.fit_transform(integer_encoded)


num_class = len(labels)
input_shape = X_train[0].shape

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255